In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, make_scorer, precision_score, recall_score, accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier


https://aws.amazon.com/blogs/machine-learning/predicting-customer-churn-with-amazon-machine-learning/

In [ ]:
!wget http://amazon-sagemaker.com/datasets/DKD2e_data_sets.zip -O DKD2e_data_sets.zip

In [ ]:
!unzip -o DKD2e_data_sets.zip

In [ ]:
data = pd.read_csv('Data sets/churn.txt')
data.head()

#### Variables del dataset
- **State:** estado de Estados Unidos en el cual el cliente reside, indicado por una abreviatura de dos caracteres; por ejemplo, OH o NJ
- **Account Length:** número de días que la cuenta ha estado activa
- **Area Code:** código de área a tres digitos correspondiente al número telefónico del cliente
- **Phone:** los 7 digitos restantes del número telefónico del cliente
- **Int’l Plan:** indica si el cliente tiene un plan para llamdas internacionales: yes/no
- **VMail Plan:** indica si el cliente tiene la funcionalidad de buzón de voz: yes/no
- **VMail Message:** promedio de mensajes de voz por mes
- **Day Mins:** número total de minutos de llamadas realizadas durante el día
- **Day Calls:** número total de llamadas realizadas durante el día
- **Day Charge:** costo facturado por las llamadas realizadas durante el día
- **Eve Mins:** número total de minutos de llamadas realizadas durante la tarde
- **Eve Calls:** número total de llamadas realizadas durante la tarde
- **Eve Charge:** costo facturado por las llamadas realizadas durante la tarde
- **Night Mins:** número total de minutos de llamadas realizadas durante la noche
- **Night Calls:** número total de llamadas realizadas durante la noche
- **Night Charge:** costo facturado por las llamadas realizadas durante la noche
- **Intl Mins:** número total de minutos de llamadas internacionales realizadas
- **Intl Calls:** número total de llamadas internacionales realizadas
- **Intl Charge:** costo facturado por las llamadas internacionales realizadas
- **CustServ Calls:** número de llamadas realizadas a Servicio al Cliente
- **Churn?:** indica si el cliente abandona el servicio o no: true/false

#### Eliminar caracteres especiales y reemplazar espacios por guiones bajos

In [ ]:
data.columns = [''.join (c if c.isalnum() else '_' for c in str(column)) for column in data.columns]
data.head()

### Análisis exploratorio del dataset

#### Selección de columnas con las que nos interesa trabajar

In [ ]:
columns = ['State', 'Account_Length', 'Area_Code', 'Int_l_Plan','VMail_Plan', 'VMail_Message', 
           'Day_Mins', 'Day_Calls','Eve_Mins', 'Eve_Calls', 'Night_Mins', 'Night_Calls', 
           'Intl_Mins', 'Intl_Calls', 'CustServ_Calls', 'Churn_']
data = data[columns]
data.head()

#### Entendiendo la distribución de las variables categóricas

In [ ]:
column = 'State'
fig1, ax = plt.subplots(1,1)
x=list(map(str,data[column].value_counts().index))
height=data[column].value_counts().values
ax.set_title(column)
ax.bar(x,height)
fig1.set_figheight(3)
fig1.set_figwidth(16)

columns = ['Area_Code', 'Int_l_Plan', 'VMail_Plan', 'Churn_']
fig2, axes = plt.subplots(1,len(columns))

for i, column in enumerate(columns, start=0):        
    ax = axes[i]
    ax.set_title(column)
    
    x=list(map(str,data[column].value_counts().index))
    height=data[column].value_counts().values
    ax.bar(x,height)
    
fig2.set_figheight(3)
fig2.set_figwidth(16)
plt.show()

#### Entendiendo la distribución de las variables numéricas

In [ ]:
columns = ['Account_Length', 'VMail_Message', 'Day_Mins', 'Day_Calls','Eve_Mins', 'Eve_Calls',
           'Night_Mins', 'Night_Calls', 'Intl_Mins', 'Intl_Calls', 'CustServ_Calls']

cols=3
fig, axes = plt.subplots(round(len(columns)/cols),cols)

for i, column in enumerate(columns, start=1): 
    ax = axes[int((i-1-((i-1)%cols))/cols), (i-1)%cols]
    ax.set_title(column)
    ax.hist(data[column])

if len(columns) < axes.shape[0]*axes.shape[1]:
    axes[-1][-1].axis('off')
    
fig.set_figheight(18)
fig.set_figwidth(16)
plt.show()

### Preparación del dataset

#### Eliminación del . al final de la palabra False o True en la columna `Churn_` y renombrarla a `Churn`

In [ ]:
data['Churn_']=data['Churn_'].str.replace('.','')
data.rename(columns={'Churn_':'Churn'}, inplace=True)
data.head()

#### One hot encoding de variables categoricas

In [ ]:
columns = ['State','Area_Code']
encoder = OneHotEncoder().fit(data[columns])

In [ ]:
transformed = encoder.transform(data[columns]).toarray()

In [ ]:
data.drop(columns,axis=1, inplace=True)
data = pd.concat([data,pd.DataFrame(transformed, columns=encoder.get_feature_names())],axis=1)
data.head()

#### Reemplazar yes/no por 1/0 en columnas Int_l_Plan y VMail_Plan

In [ ]:
data['Int_l_Plan'] = data['Int_l_Plan'].map(dict(yes=1, no=0))
data['VMail_Plan'] = data['VMail_Plan'].map(dict(yes=1, no=0))
data.head()

#### Reemplazar True/False por 1/0 en columna Churn

In [ ]:
data['Churn'] = data['Churn'].map({'True': 1, 'False': 0})

#### Separar la etiqueta o target del resto de los datos

In [ ]:
target = data[['Churn']]
data.drop(['Churn'], axis=1, inplace=True)

#### Dividir los datos para entrenamiento y validación

Verificamos que tan balanceado está nuestro dataset

In [ ]:
round(target['Churn'].value_counts(normalize=True) * 100,1)

#### Y dividimos en train (80%) y test (20%), manteniendo las mismas proporciones de observaciones por cada clase

In [ ]:
train_data, test_data, train_target, test_target = train_test_split(data, target, stratify=target, test_size=0.1)

In [ ]:
print('Train: {0} records with clasess: 0={1[0]}% and 1={1[1]}%'.format(train_target.shape[0],
                                             round(train_target['Churn'].value_counts(normalize=True) * 100, 1)))

print('Test: {0} records with clasess: 0={1[0]}% and 1={1[1]}%'.format(test_target.shape[0],
                                             round(test_target['Churn'].value_counts(normalize=True) * 100, 1)))

### Entrenamiento y selección del mejor modelo

Entrenamiento de tres modelos utilizando los siguientes algoritmos
- Gradient Boosting
- Random Forest
- Extra Trees

Para posteriormente seleccionar el mejor después de haber hecho un fine tuning de cada melo mediante un GridSearch

In [ ]:
def plot_confusion_matrix(test_set, predictions, classes, title):
    classes=np.array(classes)
    cm = confusion_matrix(test_set, predictions)
    
    fig, (ax1,ax2) = plt.subplots(1,2, gridspec_kw={'width_ratios': [2, 3]})
    im = ax1.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax1.figure.colorbar(im, ax=ax1)
    
    # We want to show all ticks...
    ax1.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title="Confusion Matrix",
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax1.get_yticklabels(), rotation=90, ha="center", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax1.text(j, i, format(cm[i, j], 'd'),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    precision = tp/(fp+tp)
    recall = tp/(fn+tp)
    specifity = tn/(tn+fp)
    
    ax2.axis('off')
    ax2.text(0,0.9, s='The overall model accuracy is {}% [ACCURACY]'.format(round(accuracy*100,2)), 
             size='12', ha='left', va='center')
    
    ax2.text(0,0.7, s='Out of the customers the model predicted as will churn, {}% will actually churn [PRECISION]'.format(round(precision*100,2)), 
             size='12', ha='left', va='center')
    
    ax2.text(0,0.5, s='The model will catch {}% of the customers who will actually churn [RECALL / SENSITIVITY]'.format(round(recall*100,2)), 
             size='12', ha='left', va='center')
    
    ax2.text(0,0.3, s='The model will catch {}% of the customers who will actually NOT churn [SPECIFITY]'.format(round(specifity*100,2)), 
             size='12', ha='left', va='center')
    
    fig.set_figheight(3)
    fig.set_figwidth(16)
    fig.suptitle(title, fontsize=16)
    plt.show()

In [ ]:
def fine_tuner(clf, dataset, param_grid, scores, splits, refit_score, title):
    skf = StratifiedKFold(n_splits=splits)    

    grid_search = GridSearchCV(clf, param_grid, scoring=scores, refit=refit_score,
                               cv=skf, return_train_score=True, n_jobs=-1)

    grid_search.fit(dataset['x_train'], dataset['y_train']['Churn'].tolist())

    predictions = grid_search.predict(dataset['x_test'])

    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)
    
    labels = ['Not Churn','Churn']
    plot_confusion_matrix(dataset['y_test'], predictions, labels, title)
    
    return grid_search

In [ ]:
def operating_point_finder(clf, data, target, min_precision):
    predict_proba = getattr(clf, 'predict_proba', None)
    if callable(predict_proba):
        predictions = predict_proba(data)[:, 1]
        
        precision, recall, thresholds = precision_recall_curve(target, predictions)

        operating_point_idx = np.argmax(precision>=min_precision)
        
        plt.figure(num=None, figsize=(9, 5), facecolor='w', edgecolor='k')
        plt.plot(recall, precision)
        plt.plot(recall[operating_point_idx], precision[operating_point_idx], 'ro', 
                 label='Decision threshold: {}'.format(thresholds[operating_point_idx]))
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend()
        plt.show()
        
        return thresholds[operating_point_idx], predictions

#### Random Forest

In [ ]:
rf_clf = RandomForestClassifier(n_jobs = -1)

parameters = {
    'min_samples_split': [3, 5, 10], 
    'n_estimators' : [100, 200, 300, 400, 500, 700],
    'max_depth': [3, 5, 15, 25],
    'max_features': [3, 5, 10, 20]
}

scores = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}

data = {
    'x_train': train_data,
    'y_train': train_target,
    'x_test': test_data,
    'y_test': test_target
}

In [ ]:
rf_clf = fine_tuner(clf=rf_clf, dataset=data, param_grid=parameters, scores=scores, splits=5, 
                    refit_score='recall_score', title='Random Forest')

In [ ]:
decision_threshold, predictions = operating_point_finder(rf_clf, test_data, test_target, 0.85)

In [ ]:
new_predictions=[1 if prediction >= decision_threshold else 0 for prediction in predictions]
labels = ['Not Churn','Churn']
plot_confusion_matrix(test_target, new_predictions, labels, 'Random Forest')

#### Gradient Boosting

In [ ]:
gb_clf = GradientBoostingClassifier()

parameters = {
    'learning_rate': [0.1, 0.05, 0.01, 0.005, 0.001],
    'n_estimators': [100, 200, 300, 400, 500, 700],
    'min_samples_split': [3, 5, 10], 
    'max_depth': [3, 5, 15, 25],
    'max_features': [3, 5, 10, 20]
}

In [ ]:
gb_clf = fine_tuner(clf=gb_clf, dataset=data, param_grid=parameters, scores=scores, splits=5, 
                    refit_score='recall_score', title='Gradient Boosting')

In [ ]:
decision_threshold, predictions = operating_point_finder(gb_clf, test_data, test_target, 0.85)

In [ ]:
new_predictions=[1 if prediction >= decision_threshold else 0 for prediction in predictions]
labels = ['Not Churn','Churn']
plot_confusion_matrix(test_target, new_predictions, labels, 'Gradient Boosting')

#### Extra Trees

In [ ]:
et_clf = ExtraTreesClassifier(n_jobs = -1)

parameters = {
    'min_samples_split': [3, 5, 10, 15], 
    'n_estimators' : [100, 200, 300, 400, 500, 700, 800],
    'max_depth': [3, 5, 15, 25],
    'max_features': [3, 5, 10, 20, 30]
}

In [ ]:
et_clf = fine_tuner(clf=et_clf, dataset=data, param_grid=parameters, scores=scores, splits=5, 
                    refit_score='recall_score', title='Extra Trees')

In [ ]:
decision_threshold, predictions = operating_point_finder(et_clf, test_data, test_target, 0.85)

In [ ]:
new_predictions=[1 if prediction >= decision_threshold else 0 for prediction in predictions]
labels = ['Not Churn','Churn']
plot_confusion_matrix(test_target, new_predictions, labels, 'Extra Trees')